In [67]:
import numpy as np
from docx import Document
from docx.shared import Pt

In [68]:
f='gstr1'
fname=f'{f}.docx'

doc=Document(fname)
all_tables=[]
for table in doc.tables:
    rows=len(table.rows)
    cols=len(table.columns)
    data=np.empty((rows,cols), dtype=object)
    for i,row in enumerate(table.rows):
        for j,cell in enumerate(row.cells):
            data[i,j]=cell.text.strip()
    all_tables.append(data)

In [69]:
def update_document(row, col, new_value, table_index):
    if row >= len(doc.tables[table_index].rows) or col >= len(doc.tables[table_index].columns):
        print("Warning: The specified cell does not exist in the table.")
        return    
    all_tables[table_index][row, col] = new_value
    cell = doc.tables[table_index].rows[row].cells[col]
    cell.text = str(new_value)
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            run.font.name, run.font.size = 'Roboto', Pt(7.5)

In [70]:
# def fn(x):
#     if x[-1][0][0].isdigit():
#         return 'yes'
#     return 'no'

# for i in range(len(all_tables)):
#     print(f'all_tables[{i}]',fn(all_tables[i]))

In [71]:
all_tables

[array([['Financial year', ''],
        ['Tax period', '']], dtype=object),
 array([['1', 'GSTIN', 'GSTIN', ''],
        ['2', '(a)', 'Legal name of the registered person', ''],
        ['2', '(b)', 'Trade name if any', ''],
        ['2', '(c)', 'ARN', ''],
        ['2', '(d)', 'ARN date', '']], dtype=object),
 array([['Description', 'No. of records', 'Document Type', 'Value (₹)',
         'Integrated Tax (₹)', 'Central Tax (₹)', 'State/UT Tax (₹)',
         'Cess (₹)'],
        ['4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
         '4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
         '4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
         '4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular',
         '4A - Taxable outward supplies made

In [72]:
# def fn(x,y,k=0):
#     if k==1:
#         c=x[-4]
#     else:
#         c=x[-1]
#     c_reshaped=c.reshape(1, -1)
#     return np.vstack((c_reshaped, y))

# all_tables[3]=fn(all_tables[2],all_tables[3])
# all_tables[4]=fn(all_tables[3],all_tables[4])
# all_tables[5]=fn(all_tables[4],all_tables[5],1)

In [73]:
def get_desc(x):
    l=[]
    for row_index, row in enumerate(x):
        try:
            z=int(row[0][0])
        except:
            z=-1
        if(z!=-1):
            l.append(row_index)
    return l

t=[]
for i in range(len(all_tables)):
    t.append(get_desc(all_tables[i]))
t

[[],
 [0, 1, 2, 3, 4],
 [1, 4, 7, 10, 15, 20],
 [1, 4, 10, 13, 16, 19, 25, 31],
 [1, 13, 20],
 [1, 7, 10]]

In [74]:
all_tables[5][10]

array(['11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendments – 11A(1), 11A(2), 11B(1), 11B(2)',
       '11(A), 11(B) - Advances received & Advances adjusted in returns comsidering Amendm

In [75]:
lists=[]
for i in range(2):
    for row_index, row in enumerate(all_tables[i]):
        for col_index, value in enumerate(row):
            if len(value)==0:
                d={}
                d['Row']=all_tables[i][row_index][-2]
                d['Value']='NaN'
                d['Column']=''
                d['Coordinates']=(row_index,col_index,i)
                lists.append(d)

In [76]:
for g in range(2,len(all_tables)):
    x=all_tables[g]
    l=t[g]
    for row_index, row in enumerate(x):
        for col_index, value in enumerate(row):
            try:
                x1=int(value.strip())
            except:
                x1=-1
            if x1!=-1 or value=='0.00':
                d={}
                d['Description']=list(set(x[max([num for num in l if num<row_index])]))[0]
                d['Row']=x[row_index,0]
                d['Value']='NaN'
                d['Coordinates']=(row_index,col_index,g)
                d['Column']=x[0,col_index]
                lists.append(d)

# for i in range(len(lists)-5, len(lists)):
#     del lists[i]['Description']

In [77]:
# import pickle
# with open('data_params_2.pkl', 'wb') as f:
#     pickle.dump(lists, f)

# with open('data_params.pkl', 'rb') as f:
#     lists=pickle.load(f)

In [78]:
# o={}
# o['Value (₹)']='Value'
# o['Integrated Tax (₹)']='IGST'
# o['Central Tax (₹)']='CGST'
# o['State/UT Tax (₹)']='SGST'
# o['Cess (₹)']='Cess'
# o['']=''

# k=[f]
# for i_index, i in enumerate(lists):
#     try:
#         s=' | '.join([f"{i_index}.",i['Row'],o[i['Column']],i['Description']])
#     except:
#         s=' | '.join([f"{i_index}.",i['Row'],o[i['Column']],''])
#     k.append(s)
# k

In [91]:
o={}
o['Value (₹)']='Value'
o['Integrated Tax (₹)']='IGST'
o['Central Tax (₹)']='CGST'
o['State/UT Tax (₹)']='SGST'
o['Cess (₹)']='Cess'
o['']=''

k=[]
for i_index, i in enumerate(lists):
    try:
        s=' | '.join([i['Row'],o[i['Column']],i['Description']])
    except:
        s=' | '.join([i['Row'],o[i['Column']],''])
    # k.append(s)
    k.append(tuple((i_index,i['Coordinates'],s)))
k

[(0, (0, 1, 0), 'Financial year |  | '),
 (1, (1, 1, 0), 'Tax period |  | '),
 (2, (0, 3, 1), 'GSTIN |  | '),
 (3, (1, 3, 1), 'Legal name of the registered person |  | '),
 (4, (2, 3, 1), 'Trade name if any |  | '),
 (5, (3, 3, 1), 'ARN |  | '),
 (6, (4, 3, 1), 'ARN date |  | '),
 (7,
  (2, 3, 2),
  'Total | Value | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular'),
 (8,
  (2, 4, 2),
  'Total | IGST | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular'),
 (9,
  (2, 5, 2),
  'Total | CGST | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular'),
 (10,
  (2, 6, 2),
  'Total | SGST | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies) - B2B Regular'),
 (11,
  (2, 7, 2),
  'Total | Cess | 4A - Taxable outward supplies made to registered persons (other than reverse charge supplies

In [92]:
import pickle
with open('gstr1_params.pkl', 'wb') as f1:
    pickle.dump(k, f1)

In [81]:
j=0
for i in lists:
    update_document(i['Coordinates'][0], i['Coordinates'][1], j, i['Coordinates'][2])
    j+=1

In [82]:
doc.save(f"{f}_Updated.docx")

In [83]:
import docx2pdf

In [84]:
docx2pdf.convert(f"{f}_Updated.docx", f"{f}.pdf")

  0%|          | 0/1 [00:00<?, ?it/s]